# Module 2 Discussion Notebook

## Revenue Optimization Scenario 1

A creative development team has three products they can sell.

A static HTML5 ad for $2,5000. We'll call this *s*.

An animated HTML5 ad for $4,000. We'll call this *a*

A dynamic HTML5 ad for $9,500. We'll call this *d*

We know we'll need a designer, a developer, and a project manager for every project. 

| Project  | Designer Hours  | Dev Hours  | PM Hours |
|----------|:---------------:|:----------:|:--------:|
| Static   | 5               | 8          | 2        | 
| Animated | 10              | 16         | 4        |
| Dynamic  | 10              | 28         | 5        |


For this upcoming quarter we know we have this many pooled hours to use per position

|Position | Hours |
|---------|:-----:|
|Designer | 425   |
|Dev      | 380   |
|PM       | 480   |

In [125]:
#Let's get started with loading up pulp and getting everything setup
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize

Now that we've got the adjustable variables built. We can start building the actual solver

We'll begin with creating the three variables for the problem and the objective function.

In [126]:
#LpMaximize your conversions
prob = LpProblem("problem", LpMaximize)

#we'll set our variables here
#lowBound=0 because we can't have negative projects
#cat=Integer because you can't sell part of a project
s = LpVariable("s", lowBound=0, cat='Integer')
a = LpVariable("a", lowBound=0, cat='Integer') 
d = LpVariable("d", lowBound=0, cat='Integer') 

#objective function here
#we're going to maximize our total revenue
prob += 2500*s + 4000*a + 9500*d

Time to fill in the constraints

In [130]:
#lets get our constraints going
#first constraint is our designer hours available
prob += 5*s + 10*a + 10*d <= 425

#second constraint is our dev hours
prob += 8*s + 16*a + 28*d <= 380

#third constraint is our pm hours
prob += 2*s + 4*a + 5*d <= 480

Run the solver + print our results

In [133]:
#solve the problem
status = prob.solve()

#check our status. 
print("Pulp Status =", LpStatus[status])

#print the results
print("Optimal # of Projects this Quarter")
print('Static Projects =' , value(s))
print('Animated Projects =' , value(a))
print('Dynamic Projects =' , value(d))


Pulp Status = Optimal
Optimal # of Projects this Quarter
Static Projects = 2
Animated Projects = 0
Dynamic Projects = 13


## Understanding the results

Based on the results, we know that there the optimal scenario involves 13 dynamic projects and 2 static projects.

Understanding this in a business context where there are a lot more variables in play including a range of hours available that are influenced by other factors, we can take this data directionally.

The creative development team needs to focus on selling the big ticket dynamic creatives. 

Any misses on dynamic sells means the fill needs to come from static projects because it takes the least amount of hours for the most revenue.

Animated projects sit in an odd middle ground and don't provide a great hours/revenue ratio. 




## Scipy edition

In [1]:
#version 1, using the version from the module 3 notebooks
#in this version there are no limits to the type of variable
#like continous or integer
#this just is continuous so you can get a non-integer answer
from scipy.optimize import linprog

#objective function
#translated from above: 2500*s + 4000*a + 9500*d
#remember we have to do f(x)*-1 because linprog only does minimize
z = [-2500, -4500, -9500] 

#coefficients of the left hand side of the inequalities
#lets get our constraints going
#first constraint is our designer hours available
#prob += 5*s + 10*a + 10*d <= 425 translated to 5,10,10 left hand side
#second constraint is our dev hours
#prob += 8*s + 16*a + 28*d <= 380 translated to 8,16,28 left hand side
#third constraint is our pm hours
#prob += 2*s + 4*a + 5*d <= 480 translated to 2,4,5 left hand side
# coefficients of the left-hand side of the inequalities
lhs = [[5, 10, 10], [8, 16, 28], [2, 4, 5]]

#coefficients of the right hand side
#top to bottom
rhs = [425, 380, 480]

#set the bounds
#(min,max) and none = to infinity
#default = (min = 0, max = infinity)
s = (0, None)
a = (0, None)
d = (0, None)

#note regarding methods, I used the default: HiGHS because other stuff is going to be deprecated 
#and to use the default, we just don't specify.
#but you should specify for best practice (be explicit)
#c=z just uses the z list to fill the objective coefficients
#A_ub coefs for the left hand side, use list 'lhs'
#b_ib coefs for the right hand side, use list 'rhs'
res = linprog(c=z, A_ub=lhs, b_ub=rhs, method='highs', bounds=(s, a, d))


#print results
print('Scipy Optimize Optimal value:', res.fun, '\n s,a,d :', res.x)
print('\n')

Scipy Optimize Optimal value: -128928.57142857142 
 s,a,d : [ 0.          0.         13.57142857]




In [1]:
#version 2, using milp in scipy
#in this version there you can set the type of variable
#integer or continous
import numpy as np
import scipy.optimize

#objective function
#translated from above: 2500*s + 4000*a + 9500*d
#remember we have to do f(x)*-1 because linprog only does minimize
z = -np.array([2500, 4500, 9500])

#coefficients of the left hand side of the inequalities
#lets get our constraints going
#first constraint is our designer hours available
#prob += 5*s + 10*a + 10*d <= 425 translated to 5,10,10 left hand side
#second constraint is our dev hours
#prob += 8*s + 16*a + 28*d <= 380 translated to 8,16,28 left hand side
#third constraint is our pm hours
#prob += 2*s + 4*a + 5*d <= 480 translated to 2,4,5 left hand side
# coefficients of the left-hand side of the inequalities
lhs = np.array([[5, 10, 10], [8, 16, 28], [2, 4, 5]])

#coefficients of the right hand side
#top to bottom
rhs = np.array([425, 380, 480])

#full_like gives us an array the same shape as rhs
#filled with negative infinity
boundlimit = np.full_like(rhs, -np.inf)

#set that we use integers only
#returns an array of 1s
integrality = np.ones_like(z)

#now that we setup the above, turn into a constraint
#this means lhs coefs = lhs
#lowerbound = boundlimit
#ubberbound = rhs
constraints = scipy.optimize.LinearConstraint(lhs, boundlimit, rhs)

#note regarding methods, I used the default: HiGHS because other stuff is going to be deprecated 
#and to use the default, we just don't specify.
#but you should specify for best practice (be explicit)
#c=z just uses the z list to fill the objective coefficients
#A_ub coefs for the left hand side, use list 'lhs'
#b_ib coefs for the right hand side, use list 'rhs'
res = scipy.optimize.milp(c=z, constraints=constraints, integrality=integrality)

#print results
print('Scipy Optimize Optimal value:', res.fun, '\n s,a,d :', res.x)
print('\n')

Scipy Optimize Optimal value: -128500.0 
 s,a,d : [ 2.  0. 13.]


